## Plots af resultater til rapport

In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

from scipy.optimize import curve_fit

In [4]:
Multiclass = pd.read_csv('/Users/sararhiger/Desktop/Master thesis /Data/Processed data/Multiclass_dataprocessed.csv')

### Performance fra binary model baseret på simuleret data for hver fejl 

In [18]:
data = {
    'Model': ["Open window", "Low setpoint", "Uncalibrated CO2 sensor", "High  setpoint"]*3,
    'metric': ['Precision']*4 + ['Recall']*4 + ['f1 score']*4,
    'mean':    [0.85, 0.97, 1, 0.86, 0.43, 0.55, 1, 0.74, 0.57, 0.7, 1, 0.78],
    'std':     [0.01, 0.03, 0, 0.10, 0.04, 0, 0, 0.24, 0.04, 0.01, 0, 0.18]
}

summary_df = pd.DataFrame(data)

summary_df

,Model,metric,mean,std
0,Open window,Precision,0.85,0.01
1,Low setpoint,Precision,0.97,0.03
2,Uncalibrated CO2 sensor,Precision,1.00,0.00
3,High setpoint,Precision,0.86,0.10
4,Open window,Recall,0.43,0.04
5,Low setpoint,Recall,0.55,0.00
6,Uncalibrated CO2 sensor,Recall,1.00,0.00
7,High setpoint,Recall,0.74,0.24
8,Open window,f1 score,0.57,0.04
9,Low setpoint,f1 score,0.70,0.01


In [19]:
# Create Plotly bar chart with custom colors
fig = px.bar(
    summary_df,
    x="Model",
    y="mean",
    color="metric",
    barmode="group",
    error_y="std",
    labels={"mean": "Score", "metric": "Metric"},
    title="Classification Performance (Mean ± Std) per Class",
    color_discrete_map={
        "Precision": "mediumseagreen",
        "Recall": "salmon",
        "f1 score": "lightblue"
    }
)

# Layout tweaks
fig.update_layout(
    yaxis_title="Performance Score %",
    uniformtext_minsize=12,
    font=dict(size=14),
    uniformtext_mode='hide',
    title_x=0.5,
    height=500
)

fig.show()

### Performance fra multiclass model baseret på simuleret data for hver fejl 

In [20]:
data = {
    'Model': ["Open window", "Low setpoint", "Uncalibrated CO2 sensor", "High  setpoint"]*3,
    'metric': ['Precision']*4 + ['Recall']*4 + ['f1 score']*4,
    'mean':    [0.85, 1, 1, 0.8, 0.45, 0.55, 1, 0.98, 0.59, 0.71, 1, 0.88],
    'std':     [0.01, 0.01, 0, 0.02, 0.02, 0, 0, 0.02, 0.02, 0, 0, 0.02]
}

summary_df = pd.DataFrame(data)

summary_df



,Model,metric,mean,std
0,Open window,Precision,0.85,0.01
1,Low setpoint,Precision,1.00,0.01
2,Uncalibrated CO2 sensor,Precision,1.00,0.00
3,High setpoint,Precision,0.80,0.02
4,Open window,Recall,0.45,0.02
5,Low setpoint,Recall,0.55,0.00
6,Uncalibrated CO2 sensor,Recall,1.00,0.00
7,High setpoint,Recall,0.98,0.02
8,Open window,f1 score,0.59,0.02
9,Low setpoint,f1 score,0.71,0.00


In [21]:
# Create Plotly bar chart with custom colors
fig = px.bar(
    summary_df,
    x="Model",
    y="mean",
    color="metric",
    barmode="group",
    error_y="std",
    labels={"mean": "Score", "metric": "Metric"},
    title="Classification Performance (Mean ± Std) per Class",
    color_discrete_map={
        "Precision": "mediumseagreen",
        "Recall": "salmon",
        "f1 score": "lightblue"
    }
)

# Layout tweaks
fig.update_layout(
    yaxis_title="Performance Score %",
    uniformtext_minsize=12,
    font=dict(size=14),
    uniformtext_mode='hide',
    title_x=0.5,
    height=500
)

fig.show()

## Plot with temp and energy with all faults implemented (Multiclass)

In [7]:
df_simulated = Multiclass

In [8]:
fig = go.Figure()

# Plot the temperature curve
fig.add_trace(go.Scatter(
    x=df_simulated['Time'], 
    y=df_simulated['Mean air temperature, degC, zone 8'], 
    mode='lines', 
    name='Mean air temperature',
    yaxis='y1'
))

# Add heating consumption on second y-axis as a dotted line
fig.add_trace(go.Scatter(
    x=df_simulated['Time'],
    y=df_simulated['Local heating units, W, zone 8'],  # Replace with your actual column name
    mode='lines',
    name='Normalized heating consumption',
    line=dict(dash='dot'),
    yaxis='y2'
))

# Ensure 'Time' is datetime
#df_simulated['Time'] = pd.to_datetime(df_simulated['Time'])

# Get one timestamp per day (midnight)
#tick_vals = df_simulated['Time'].dt.normalize().drop_duplicates().sort_values()

# Update layout
fig.update_layout(
    title="temp and heating consumption multiclass",
    xaxis_title="Time",
    yaxis=dict(
        title="Temperature (°C)",
        side="left"
    ),
    legend=dict(
        title="Legend",
        x=1.08,
        xanchor='left'
    ),
    yaxis2=dict(
        title="Heating Consumption (Normalized)",  # Update unit as needed
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend_title="Legend",
    template="plotly",
    width=1200,
    height = 500
    #xaxis=dict(
        #tickmode='array',
        #tickvals=tick_vals,
        #tickformat="%b %d"
    )


fig.show()

### Performance VS implemted faults 

In [9]:
df = pd.DataFrame({
    "Number of faults simulated": ["1", "3", "7", "15"],
    "Precision [%]": [0 , 1 , 0.83 , 0.86], 
    "Recall [%]": [0 , 0.15 , 0.38 , 0.46],
    "f1 score [%]": [0.02 , 0.37 , 0.52 , 0.57], 
    "Faults detcted in test zone [%]": [0 , 0.25 , 0.5 , 0.5],
})

# Display the DataFrame
print(df)

  Number of faults simulated  Precision [%]  Recall [%]  f1 score [%]  \
0                          1           0.00        0.00          0.02   
1                          3           1.00        0.15          0.37   
2                          7           0.83        0.38          0.52   
3                         15           0.86        0.46          0.57   

   Faults detcted in test zone [%]  
0                             0.00  
1                             0.25  
2                             0.50  
3                             0.50  


In [10]:
import plotly.graph_objects as go

fig = go.Figure()

# Add the f1 score line
fig.add_trace(go.Scatter(
    x=df["Number of faults simulated"],
    y=df["f1 score [%]"],
    mode='lines+markers',
    name='f1 score [%]',
    showlegend=True
))

# Update layout with custom fonts and axis ticks
fig.update_layout(
    title="Performance Metrics vs. Number of Faults Simulated",
    title_font=dict(size=16),  # Larger title font
    xaxis_title="Number of faults simulated during a year",
    yaxis_title="Percentage",
    font=dict(size=16),  # Base font
    xaxis=dict(
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title_font=dict(size=16),
        tickfont=dict(size=16),
        range=[0, 1],
        tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1],
        #ticktext=["0", "20%", "40%", "60%", "80%", "100%"]
    ),
    legend=dict(
        x=0.01, y=0.99,
        bgcolor="white",
        bordercolor="Black",
        borderwidth=1,
        font=dict(size=16)  # Legend font
    ),
    width=900,
    height=500,
    template="plotly"
)

fig.show()


## Model improvements

In [11]:
df = pd.DataFrame({
    "Improvement": ["Baseline", "+Normalized heating consumption", "+Outdoor temperature", "+Global horizontal irradiance"],
    "f1 score [%]": [0.22 , 0.71 , 0.84 , 0.87], 
})

fig = go.Figure()

# Add the f1 score line
fig.add_trace(go.Scatter(
    x=df["Improvement"],
    y=df["f1 score [%]"],
    mode='lines+markers',
    name='f1 score [%]',
    showlegend=True
))

fig.update_layout(
    title="Model improvement with additional features",
    xaxis_title="Added ",
    yaxis_title="Percentage",
    width=900,
    height=500,
    template="plotly",
    title_font=dict(size=14),  # Larger title font
    font=dict(size=14),  # Base font
    yaxis=dict(
        range=[0, 1],         # y-axis from 0 to 1
        tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1],
        title_font=dict(size=16),
        tickfont=dict(size=16),  # Set ticks at 0.2 intervals
        #ticktext=["0", "20%", "40%", "60%", "80%", "100%"]  # Optional: Custom text for ticks
    ),
    legend=dict(
        x=0.01, y=0.99,
        bgcolor="white",
        bordercolor="Black",
        borderwidth=1,
        font=dict(size=16)
    )
)

fig.show()

## Measured FDD performance multi vs binary

In [12]:

# Your data
data = {
    'Model': ["Open window ", "High setpoint ", "Open window", "High setpoint"]*3,
    'metric': ['Precision']*4 + ['Recall']*4 + ['f1 score']*4,
    'mean':    [97, 100, 90, 0, 11, 78, 10, 0, 19, 85, 18, 0],
    'std':     [11, 0, 16, 0, 0, 22, 2, 0, 0, 17, 3, 0],
    'Group': ["Binary", "Binary", "Multiclass", "Multiclass"]*3
}

measured_df = pd.DataFrame(data)

measured_df

,Model,metric,mean,std,Group
0,Open window,Precision,97,11,Binary
1,High setpoint,Precision,100,0,Binary
2,Open window,Precision,90,16,Multiclass
3,High setpoint,Precision,0,0,Multiclass
4,Open window,Recall,11,0,Binary
5,High setpoint,Recall,78,22,Binary
6,Open window,Recall,10,2,Multiclass
7,High setpoint,Recall,0,0,Multiclass
8,Open window,f1 score,19,0,Binary
9,High setpoint,f1 score,85,17,Binary


In [13]:

fig = px.bar(
    measured_df,
    x="Model",
    y="mean",
    color="metric",
    barmode="group",
    error_y="std",
    labels={"mean": "Score", "metric": "Metric"},
    title="Classification Performance (Mean ± Std) per Class",
    color_discrete_map={
        "Precision": "mediumseagreen",
        "Recall": "salmon",
        "f1 score": "lightblue"
    }
)

# Layout tweaks
fig.update_layout(
    yaxis_title="Performance Score %",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    title_x=0.5,
    width=1200,
    height=500, 
    xaxis_title=None,
    bargroupgap=0.1
)

fig.add_annotation(text="Binary",
                   x=0.22, y=-0.15,
                   xref="paper", yref="paper",
                   showarrow=False,
                   font=dict(size=16, color="black"))

fig.add_annotation(text="Multiclass",
                   x=0.795, y=-0.15,
                   xref="paper", yref="paper",
                   showarrow=False,
                   font=dict(size=16, color="black"))

fig.show()